## Objective

The objective of this experiment is to evaluate and compare the performance of three distinct Combinatorial Bayesian Optimization (CBO) strategies for a scheduling problem, specifically outpatient appointment scheduling as described by Kaandorp and Koole (2007). We investigate:

1.  CBO utilizing Expected Improvement (EI) as the acquisition function.
2.  CBO utilizing Lower Confidence Bound (LCB) as the acquisition function with a fixed kappa ($\kappa$) value.
3.  CBO utilizing Lower Confidence Bound (LCB) as the acquisition function with a dynamically increasing kappa ($\kappa$) value.

We aim to determine which strategy is most effective in identifying an optimal or near-optimal schedule, as measured by the objective function value, leveraging dictionary-based embeddings for the high-dimensional combinatorial space (Deshwal et al., 2023).

## Background

We consider an outpatient appointment scheduling problem (Kaandorp and Koole, 2007) where the schedule is represented by a vector $\mathbf{x} = (x_0, x_1, \ldots, x_{T-1})^T$. This vector comprises $T$ components, where $x_j$ denotes the non-negative allocation (number of patients) to time slot $j$, for $j = 0, \ldots, T-1$. A fundamental constraint is that the total allocation across all time slots must equal a fixed constant $N$: $$\sum_{j=0}^{T-1} x_j = N$$ We require $x_j \ge 0$ for all $j = 0, \ldots, T-1$. Consequently, a valid schedule $\mathbf{x}$ belongs to the feasible set $\mathcal{F} = \{ \mathbf{z} \in \mathbb{D}^{T} \mid \sum_{j=0}^{T-1} z_j = N, z_j \ge 0 \text{ for all } j\}$, where $\mathbb{D}$ is the set of non-negative integers ($\mathbb{Z}_{\ge 0}$).

Kaandorp and Koole (2007) define a neighborhood structure for local search based on perturbation vectors derived from a set of $T$ basis change vectors, $v_i \in \mathbb{D}^{T}$, for $i = 0, \ldots, T-1$. These basis vectors represent elementary shifts of allocation between time slots:

-   $v_0 = (-1, 0, \ldots, 0, 1)$ (Shift unit *from* slot 0 *to* slot $T-1$)
-   $v_1 = (1, -1, 0, \ldots, 0)$ (Shift unit *from* slot 1 *to* slot 0)
-   $v_i = (0, \ldots, 0, \underbrace{1}_{\text{pos } i-1}, \underbrace{-1}_{\text{pos } i}, 0, \ldots, 0)$ for $i = 2, \ldots, T-1$ (Shift unit *from* slot $i$ *to* slot $i-1$)

A key property of these basis vectors is that the sum of components for each vector is zero: $\sum_{j=0}^{T-1} v_{ij} = 0$ for all $i=0, \ldots, T-1$.

Perturbations are constructed using a binary selection vector $\mathbf{U} = (u_0, u_1, \ldots, u_{T-1})$, where $u_i \in \{0, 1\}$. Each $u_i$ indicates whether the basis change $v_i$ is included in the perturbation. The resulting perturbation vector $\mathbf{r}(\mathbf{U}) \in \mathbb{D}^{T}$ is the linear combination: $$\mathbf{r}(\mathbf{U}) := \sum_{i=0}^{T-1} u_i v_i$$

Since each $v_i$ sums to zero, any perturbation $\mathbf{r}(\mathbf{U})$ also sums to zero: $\sum_{j=0}^{T-1} r_j(\mathbf{U}) = 0$. This ensures that applying such a perturbation to a valid schedule $\mathbf{x}$ preserves the total allocation $N$.

The neighborhood of a schedule $\mathbf{x} \in \mathcal{F}$, denoted by $\mathcal{N}(\mathbf{x})$, comprises all distinct, feasible schedules $\mathbf{x}'$ reachable by applying a non-zero perturbation $\mathbf{r}(\mathbf{U})$ (Kaandorp and Koole, 2007, use a slightly different but related neighborhood definition based on combinations of these basis vectors).

The objective function to be minimized is a weighted sum of Expected Waiting Time (EWT) and Expected Staff Penalty (ESP), as defined by Kaandorp and Koole (2007):
$$C(\mathbf{x}) = w \cdot EWT(\mathbf{x}) + (1-w) \cdot ESP(\mathbf{x})$$
Kaandorp and Koole (2007) prove that this objective function is multimodular, which guarantees that a local search algorithm using their defined neighborhood converges to the global optimum.

However, evaluating $C(\mathbf{x})$ can be computationally expensive, especially for large $N$ and $T$. Furthermore, the search space defined by the binary vectors $\mathbf{U}$ is high-dimensional ($2^T - 2$ possibilities, excluding $\mathbf{0}$ and $\mathbf{1}$). Bayesian Optimization (BO) is a suitable framework for optimizing such expensive black-box functions. Standard BO methods often struggle with high-dimensional combinatorial spaces. Deshwal et al. (2023) propose a method using dictionary-based embeddings (Hamming Embedding via Dictionaries - HED) to map the high-dimensional binary space of $\mathbf{U}$ vectors into a lower-dimensional continuous space, where standard Gaussian Process (GP) models can be effectively applied. This experiment applies the HED approach within a BO framework to solve the scheduling problem formulated by Kaandorp and Koole (2007).

## Hypothesis

We hypothesize that:

1.  All three CBO strategies, leveraging the HED embedding (Deshwal et al., 2023), will be capable of finding schedules superior to the initial schedule derived from the Bailey-Welch method (a common heuristic mentioned by Kaandorp and Koole (2007)).
2.  CBO strategies employing Lower Confidence Bound (LCB) may exhibit superior performance or faster convergence compared to Expected Improvement (EI), due to the explicit exploration-exploitation trade-off inherent in LCB.
3.  The LCB strategy with a dynamically increasing kappa ($\kappa$) value will demonstrate a more robust search, potentially leading to improved final solutions compared to a fixed $\kappa$.

## Methodology

### Tools and Materials

-   Programming Language: Python 3
-   Core Libraries: NumPy, SciPy
-   Machine Learning: Scikit-learn (for `GaussianProcessRegressor`, `MinMaxScaler`)
-   Data Structures: Standard Python lists and dictionaries, NumPy arrays.
-   Imported functions: `bailey_welch_schedule`, `get_v_star`, `compute_convolutions`, `calculate_objective_serv_time_lookup` (implementing the logic from Kaandorp and Koole (2007), assumed to be in an external `functions.py` file).

### Experimental Design

Three distinct Bayesian optimization experiments are conducted, applying the HED embedding approach (Deshwal et al., 2023) to the scheduling problem:

1.  **Experiment 1: Expected Improvement (EI)**
    * Acquisition Function: Expected Improvement.
    * Objective: Minimize $C(\mathbf{x})$ by iteratively selecting candidate vectors $\mathbf{U}$ (via their embeddings) that maximize the EI.
2.  **Experiment 2: Lower Confidence Bound (LCB) - Fixed Kappa**
    * Acquisition Function: Lower Confidence Bound.
    * Objective: Minimize $C(\mathbf{x})$ using a fixed `kappa` ($\kappa$) value in the LCB acquisition function applied to the GP model over the embedded space.
3.  **Experiment 3: Lower Confidence Bound (LCB) - Increasing Kappa**
    * Acquisition Function: Lower Confidence Bound.
    * Objective: Minimize $C(\mathbf{x})$ using a `kappa` ($\kappa$) value that dynamically increases whenever an improved solution is discovered during the BO process.

For all experiments, Hamming Distance Embedding (HED) with a "diverse random" dictionary construction strategy (Deshwal et al., 2023) is employed to map the binary perturbation vectors $\mathbf{U}$ to a continuous embedding space. A Gaussian Process (GP) model with Automatic Relevance Determination (ARD) kernels models the (negative) objective function in this embedded space.

### Variables

-   **Independent Variables**:
    -   Type of acquisition function (EI, LCB).
    -   Strategy for $\kappa$ in LCB (fixed, increasing).
    -   The specific binary perturbation vector $\mathbf{U}$ selected in each iteration (chosen via optimizing the acquisition function over the embedded space).
-   **Dependent Variables**:
    -   The objective function value $C(\mathbf{x}')$ for the resulting schedule $\mathbf{x}' = \mathbf{x} + \mathbf{r}(\mathbf{U})$ (calculated using the method from Kaandorp and Koole (2007)).
    -   The best objective function value found throughout the optimization process.

### Data Collection

Data, comprising evaluated pairs $(\mathbf{U}, C(\mathbf{x}'))$, is collected iteratively:

-   An initial set of `N_INITIAL` randomly generated $\mathbf{U}$ vectors is evaluated.
-   In each of the subsequent `N_ITERATIONS`, `BATCH_SIZE_q` new $\mathbf{U}$ vectors are selected by optimizing the respective acquisition function over `NUM_CANDIDATES_Acqf` randomly generated candidate vectors in the original binary space (evaluated via their embeddings). These newly selected vectors are then evaluated, and the results are added to the dataset.

### Sample Size and Selection

-   **N_INITIAL**: 20 (number of initial random evaluations)
-   **N_ITERATIONS**: 25 (number of Bayesian optimization iterations)
-   **BATCH_SIZE_q**: 5 (number of candidates selected and evaluated per iteration)
-   **NUM_CANDIDATES_Acqf**: $T \times 1024 = 20 \times 1024 = 20480$ (number of random candidates generated for optimizing the acquisition function in each iteration)
-   **m**: 128 (dimensionality of the HED embedding space, following Deshwal et al. (2023))

The selection of new points for evaluation is guided by the respective acquisition function (EI or LCB) optimized over the embedded space representation of candidate $\mathbf{U}$ vectors.

### Experimental Procedure

#### 1. Setup
Import necessary libraries and configure warning filters.

```python
# Core Libraries
import numpy as np
import time
import warnings
from scipy.optimize import minimize
from typing import List, Dict, Tuple, Callable, Optional, Union, Any, Iterable

# Scikit-learn for GP, Scaling, and potentially acquisition functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, ConstantKernel, WhiteKernel
from sklearn.preprocessing import MinMaxScaler
from sklearn.exceptions import ConvergenceWarning

# SciPy for statistics (needed for Expected Improvement calculation)
from scipy.stats import norm

# Assume functions.py contains implementations based on Kaandorp and Koole (2007)
# from functions import bailey_welch_schedule, get_v_star, compute_convolutions, calculate_objective_serv_time_lookup
# Mocking them for now as the file is not provided:
def bailey_welch_schedule(T, d, N, s): 
    # Placeholder: Returns a simple initial schedule (e.g., even distribution or zeros)
    # Kaandorp and Koole (2007) mention Bailey-Welch schedules two patients at t=0, then one per block.
    print(f"Mock bailey_welch_schedule called with T={T}, d={d}, N={N}")
    mock_schedule = np.zeros(T)
    if T > 0 and N > 0:
        mock_schedule[0] = 2 # Bailey-Welch starts with two
        remaining_N = N - 2
        intervals_per_patient = T / N # Approximate spacing
        current_interval = intervals_per_patient 
        for _ in range(remaining_N):
            idx = min(int(round(current_interval)), T-1)
            # Ensure we don't overwrite the first slot if T is small
            if idx == 0 and mock_schedule[0] == 2: 
                 idx = 1 % T # Put in next slot if available
            mock_schedule[idx] += 1
            current_interval += intervals_per_patient
        # Adjust if sum is not N due to rounding/integer constraints
        current_sum = np.sum(mock_schedule)
        diff = N - current_sum
        if diff > 0: # Add remaining patients somewhere
            for i in range(int(diff)): mock_schedule[(i+1)%T] += 1
        elif diff < 0: # Remove excess patients (less likely with this mock)
             for i in range(int(abs(diff))): 
                 non_zero_indices = np.where(mock_schedule > 0)[0]
                 if len(non_zero_indices) > 0:
                     mock_schedule[non_zero_indices[-1]] -= 1 # Remove from later slots first
                 else: break # Should not happen if N > 0
    return mock_schedule


def get_v_star(T): 
    # Basis change vectors as defined in Kaandorp and Koole (2007), Appendix A
    # Note: Their v_i corresponds to u_i in the original code's notation
    print(f"Mock get_v_star called with T={T}")
    v_star_matrix = np.zeros((T, T))
    if T > 0:
        # v_0 = (-1, 0, ..., 0, 1) -> Corresponds to u_1 in Kaandorp&Koole notation (shift from t=1 to t=T)
        # The code's v_0 shifts from slot 0 to T-1. Let's match the code's definition.
        v_star_matrix[0, 0] = -1
        v_star_matrix[0, T-1] = 1 
        # v_1 = (1, -1, 0, ..., 0) -> Corresponds to u_2 in Kaandorp&Koole (shift from t=2 to t=1)
        # Matches code's v_1 (shift from slot 1 to 0)
        if T > 1:
            v_star_matrix[1, 1] = -1
            v_star_matrix[1, 0] = 1
        # v_i = (0,..,1_{i-1},-1_i,..,0) for i=2..T-1 -> Corresponds to u_{i+1} in Kaandorp&Koole (shift from t=i+1 to t=i)
        # Matches code's v_i (shift from slot i to i-1)
        for i in range(2, T):
            v_star_matrix[i, i] = -1
            v_star_matrix[i, i-1] = 1
    # The provided code uses T basis vectors. Kaandorp&Koole define T basis vectors u_1..u_T
    # Let's ensure the dimensions match the code (TxT matrix for T basis vectors)
    return v_star_matrix # Returns a TxT matrix where row i is basis vector v_i

def compute_convolutions(s, N, q_prob): 
    # Placeholder: Computes convolutions for queueing calculations (Kaandorp and Koole, 2007)
    print(f"Mock compute_convolutions called with N={N}, q_prob={q_prob}")
    return {"mock_convolution_data": True} 

def calculate_objective_serv_time_lookup(Y, d, convolutions): 
    # Placeholder: Calculates EWT and ESP based on schedule Y (Kaandorp and Koole, 2007)
    # Simple mock: return sum of Y as ewt and variance as esp, scaled
    mock_ewt = np.sum(Y) * 0.1 * d # Factor in interval duration
    mock_esp = np.var(Y) * 0.5 * d
    return mock_ewt, mock_esp

# Filter warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=ConvergenceWarning) 
```

#### 2. Constants
Definition of problem parameters and initial configuration.

```python
# --- Problem Definition ---

# Fixed Data (Parameters for the scheduling problem based on Kaandorp and Koole (2007))
N = 21          # Total number of patients to be scheduled
T = 20          # Number of time slots (also dimension of perturbation vector U)
d = 5           # Duration of each time slot (e.g., in minutes)
max_s = 20      # Maximum possible service time (used for generating distribution s_dist)
q_prob = 0.20   # No-show probability (denoted 'rho' in Kaandorp and Koole (2007))
w = 0.1         # Weight for EWT in objective C(x)
l_target = 10   # Target average service time (parameter 'l' in original code)
v_star = get_v_star(T) # Basis change vectors matrix (T x T)

# Function to generate a weighted service time probability distribution
def generate_weighted_list(max_s_val: int, target_l: float, split_i: int) -> Optional[np.ndarray]:
    """
    Generates a service time probability distribution s.
    (Implementation as provided previously)
    """
    # Input Validation
    if not isinstance(max_s_val, int) or max_s_val <= 0: return None
    if not isinstance(target_l, (int, float)) or not (1 <= target_l <= max_s_val): return None
    if not isinstance(split_i, int) or not (0 < split_i < max_s_val): return None

    def _objective(p_dist: np.ndarray) -> float:
        service_times = np.arange(1, max_s_val + 1)
        weighted_avg = np.dot(service_times, p_dist)
        return (weighted_avg - target_l) ** 2

    constraints = ({'type': 'eq', 'fun': lambda p_dist: np.sum(p_dist) - 1.0})
    bounds = [(0, 1)] * max_s_val
    initial_guess = np.random.dirichlet(np.ones(max_s_val))
    
    try:
        result = minimize(_objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)
        if not result.success: return None
        optimized_probs = result.x
        optimized_probs[optimized_probs < 0] = 0
        current_sum = np.sum(optimized_probs)
        if not np.isclose(current_sum, 1.0):
            if current_sum > 0: optimized_probs /= current_sum
            else: return None
    except Exception as e: return None

    first_part_probs = optimized_probs[:split_i]
    second_part_probs = optimized_probs[split_i:]
    sorted_first_part = np.sort(first_part_probs)      
    sorted_second_part = np.sort(second_part_probs)[::-1] 
    
    final_dist = np.zeros(max_s_val + 1)
    final_dist[1 : split_i + 1] = sorted_first_part
    final_dist[split_i + 1 : max_s_val + 1] = sorted_second_part
    
    if not np.isclose(np.sum(final_dist[1:]), 1.0): print(f"Warning: Service dist sum {np.sum(final_dist[1:])}")
    return final_dist

# Generate service time distribution 's_dist'
i_val = 5 
s_dist = generate_weighted_list(max_s, l_target, i_val) 
if s_dist is not None:
    convolutions = compute_convolutions(s_dist, N, q_prob)
else:
    print("Failed to generate service time distribution. Using fallback.")
    s_dist = np.zeros(max_s + 1); 
    if max_s > 0: s_dist[1:] = 1/max_s 
    convolutions = compute_convolutions(s_dist, N, q_prob) 

# Initial schedule 'X_initial_schedule' from Bailey-Welch heuristic
X_initial_schedule = np.array(bailey_welch_schedule(T, d, N, s_dist))
print(f"Initial schedule X: {X_initial_schedule}")

# Penalty for infeasible solutions
LARGE_PENALTY = 1e10
```

#### 3. Common Functions (Objective Evaluation and HED)
Objective evaluation implements $C(\mathbf{x})$ from Kaandorp and Koole (2007). HED implementation follows Deshwal et al. (2023).

```python
def evaluate_objective(U_np, X_vec, v_star_matrix, convolutions_data, d_interval, w_weight):
    """
    Evaluates the objective function C(x') where x' = X_vec + r(U_np).
    Uses logic from Kaandorp and Koole (2007) via calculate_objective_serv_time_lookup.
    Applies HED approach from Deshwal et al. (2023) by operating on U_np.
    (Implementation as provided previously)
    """
    # Input validation...
    if not isinstance(U_np, np.ndarray): raise TypeError("Input U_np must be a numpy array")
    if U_np.ndim != 1: raise ValueError("Input U_np must be 1-dimensional")
    if U_np.shape[0] != v_star_matrix.shape[0]: raise ValueError(f"Dimension mismatch: U_np length {U_np.shape[0]} != v_star_matrix rows {v_star_matrix.shape[0]}.")
    if X_vec.shape[0] != v_star_matrix.shape[1]: raise ValueError("Dimension mismatch: X_vec length must match v_star_matrix columns.")
    if not np.all((U_np == 0) | (U_np == 1)): raise ValueError("Input U_np must be binary (0s and 1s).")

    perturbation_rU = np.sum(v_star_matrix[U_np == 1, :], axis=0)
    Y_schedule = X_vec + perturbation_rU

    if np.all(Y_schedule >= 0):
        ewt, esp = calculate_objective_serv_time_lookup(Y_schedule, d_interval, convolutions_data)
        objective_value = w_weight * ewt + (1 - w_weight) * esp
        return objective_value
    else:
        return LARGE_PENALTY

# --- Hamming Distance Embedding (HED) Implementation (Deshwal et al., 2023) ---
def hamming_distance(u1_vec: np.ndarray, u2_vec: np.ndarray) -> int:
    """Calculates the Hamming distance."""
    return np.sum(u1_vec != u2_vec)

def generate_diverse_random_dictionary(T_dim: int, m_embed_dim: int) -> np.ndarray:
    """Generates the random dictionary A using the 'diverse random' strategy."""
    dictionary_A = np.zeros((m_embed_dim, T_dim), dtype=int)
    for i in range(m_embed_dim):
        theta = np.random.uniform(0, 1) 
        row = (np.random.rand(T_dim) < theta).astype(int)
        dictionary_A[i, :] = row
    return dictionary_A

def embed_vector(U_binary_vec: np.ndarray, dictionary_A: np.ndarray) -> np.ndarray:
    """Embeds a single binary vector U_binary_vec using HED."""
    m_embed_dim = dictionary_A.shape[0]
    embedding_phi = np.zeros(m_embed_dim, dtype=float) 
    for i in range(m_embed_dim):
        embedding_phi[i] = hamming_distance(U_binary_vec, dictionary_A[i, :])
    return embedding_phi

def embed_batch(U_batch_binary_vecs: np.ndarray, dictionary_A: np.ndarray) -> np.ndarray:
    """Embeds a batch of binary vectors U_batch_binary_vecs using HED."""
    m_embed_dim = dictionary_A.shape[0]
    if U_batch_binary_vecs.ndim == 1: U_batch_binary_vecs = U_batch_binary_vecs.reshape(1, -1)
    batch_size = U_batch_binary_vecs.shape[0]
    embeddings_np = np.zeros((batch_size, m_embed_dim), dtype=float) 
    for j in range(batch_size):
        embeddings_np[j, :] = embed_vector(U_batch_binary_vecs[j, :], dictionary_A)
    return embeddings_np

# --- Common Gaussian Process (GP) Fitting Function (Deshwal et al., 2023) ---
def get_fitted_model(train_X_embedded_scaled: np.ndarray, train_Y_neg_obj: np.ndarray, m_embed_dim: int) -> GaussianProcessRegressor:
    """Fits a GP model to the scaled embedded training data."""
    if train_Y_neg_obj.ndim > 1 and train_Y_neg_obj.shape[1] == 1: train_Y_neg_obj = train_Y_neg_obj.ravel()

    kernel = ConstantKernel(1.0, constant_value_bounds=(1e-3, 1e3)) * \
             Matern(length_scale=np.ones(m_embed_dim), length_scale_bounds=(1e-2, 1e2), nu=2.5) + \
             WhiteKernel(noise_level=1e-10, noise_level_bounds="fixed") 
    
    gp_model = GaussianProcessRegressor(kernel=kernel, alpha=1e-10, n_restarts_optimizer=10, random_state=42)
    gp_model.fit(train_X_embedded_scaled, train_Y_neg_obj)
    return gp_model
```

#### 4. Experiment 1: CBO with Expected Improvement (EI)
Applies the methodology from Deshwal et al. (2023) using EI.

```python
# --- EI Specific Acquisition Function and Optimization ---
def expected_improvement(mu_pred: np.ndarray, sigma_pred: np.ndarray, f_best_neg: float, xi: float = 0.01) -> np.ndarray:
    """Computes the Expected Improvement (EI) acquisition function."""
    sigma_pred = np.maximum(sigma_pred, 1e-9)
    Z = (mu_pred - f_best_neg - xi) / sigma_pred
    ei_values = (mu_pred - f_best_neg - xi) * norm.cdf(Z) + sigma_pred * norm.pdf(Z)
    ei_values[sigma_pred <= 1e-9] = 0.0
    return ei_values

def optimize_acqf_ei(gp_model: GaussianProcessRegressor, scaler: MinMaxScaler, 
                     dictionary_A: np.ndarray, T_dim: int, q_batch_size: int, 
                     num_acqf_candidates: int, current_best_neg_f_val: float) -> np.ndarray:
    """Optimizes the EI acquisition function using HED."""
    m_embed_dim = dictionary_A.shape[0]
    candidate_u_vectors_np = np.random.randint(0, 2, size=(num_acqf_candidates, T_dim))
    embedded_candidates_np = embed_batch(candidate_u_vectors_np, dictionary_A)
    embedded_candidates_scaled_np = scaler.transform(embedded_candidates_np)
    mu_pred, std_pred = gp_model.predict(embedded_candidates_scaled_np, return_std=True)
    acq_values = expected_improvement(mu_pred, std_pred, current_best_neg_f_val, xi=0.01)
    top_indices = np.argsort(acq_values)[-q_batch_size:][::-1]
    return candidate_u_vectors_np[top_indices, :]

# --- BO Loop Parameters ---
N_INITIAL_POINTS = 20
N_BO_ITERATIONS = 25
BATCH_SIZE_Q = 5 
NUM_ACQF_CANDIDATES = T * 1024 
M_EMBED_DIM = 128 # Dictionary size as used in Deshwal et al. (2023) experiments

# --- EI BO Loop ---
print("\n--- Starting Experiment 1: CBO with Expected Improvement (EI) ---")
evaluated_U_np_list_ei = []  
evaluated_f_vals_ei = []     
train_Y_list_ei = []         

# Initialization 
print(f"Generating {N_INITIAL_POINTS} initial random points for EI...")
initial_candidates_ei = []
while len(initial_candidates_ei) < N_INITIAL_POINTS:
    U_init_candidate = np.random.randint(0, 2, size=T)
    if not any(np.array_equal(U_init_candidate, u) for u in initial_candidates_ei):
        initial_candidates_ei.append(U_init_candidate)

for U_init in initial_candidates_ei:
    f_val = evaluate_objective(U_init, X_initial_schedule, v_star, convolutions, d, w)
    neg_f_val = -f_val 
    evaluated_U_np_list_ei.append(U_init)
    evaluated_f_vals_ei.append(f_val)
    train_Y_list_ei.append(neg_f_val)

best_obj_so_far_ei = min(evaluated_f_vals_ei) if evaluated_f_vals_ei else float('inf')
print(f"Initial best objective value (EI): {best_obj_so_far_ei:.4f}")
if not np.isfinite(best_obj_so_far_ei): print("Warning: Initial best objective (EI) is infinite.")

# Bayesian Optimization Iterations
for iteration in range(N_BO_ITERATIONS):
    iter_start_time = time.time()
    print(f"\n--- EI Iteration {iteration + 1}/{N_BO_ITERATIONS} ---")
    current_dictionary_A_ei = generate_diverse_random_dictionary(T, M_EMBED_DIM)
    if not evaluated_U_np_list_ei: continue
    evaluated_U_np_array_ei = np.array(evaluated_U_np_list_ei)
    embedded_train_X_ei = embed_batch(evaluated_U_np_array_ei, current_dictionary_A_ei)
    scaler_ei = MinMaxScaler() 
    if embedded_train_X_ei.shape[0] > 0:
        embedded_train_X_scaled_ei = scaler_ei.fit_transform(embedded_train_X_ei)
    else: embedded_train_X_scaled_ei = embedded_train_X_ei
    train_Y_for_fit_ei = np.array(train_Y_list_ei) 
    if embedded_train_X_scaled_ei.shape[0] > 0 and train_Y_for_fit_ei.shape[0] == embedded_train_X_scaled_ei.shape[0]:
        gp_model_ei = get_fitted_model(embedded_train_X_scaled_ei, train_Y_for_fit_ei, M_EMBED_DIM)
    else: continue
    current_best_neg_f_val_ei = np.max(train_Y_for_fit_ei) if train_Y_for_fit_ei.size > 0 else -float('inf')
    if current_best_neg_f_val_ei <= -LARGE_PENALTY / 2 and np.isfinite(current_best_neg_f_val_ei): pass
    next_U_candidates_np_ei = optimize_acqf_ei(
        gp_model_ei, scaler_ei, current_dictionary_A_ei, T, BATCH_SIZE_Q, NUM_ACQF_CANDIDATES, current_best_neg_f_val_ei
    )
    for i_cand in range(next_U_candidates_np_ei.shape[0]):
        next_U = next_U_candidates_np_ei[i_cand, :]
        if any(np.array_equal(next_U, u) for u in evaluated_U_np_list_ei): continue
        next_f = evaluate_objective(next_U, X_initial_schedule, v_star, convolutions, d, w)
        evaluated_U_np_list_ei.append(next_U)
        evaluated_f_vals_ei.append(next_f)
        train_Y_list_ei.append(-next_f) 
        if next_f < best_obj_so_far_ei: best_obj_so_far_ei = next_f
    print(f"Best objective (EI) so far: {best_obj_so_far_ei:.4f}")

# --- Results for EI ---
print("\n--- EI Optimization Finished ---")
if not evaluated_f_vals_ei:
    final_best_f_ei = float('inf'); final_best_U_ei = None
else:
    final_best_idx_ei = np.argmin(evaluated_f_vals_ei)
    final_best_U_ei = evaluated_U_np_list_ei[final_best_idx_ei]
    final_best_f_ei = evaluated_f_vals_ei[final_best_idx_ei]
    print(f"Total evaluations (EI): {len(evaluated_f_vals_ei)}")
    print(f"Best Objective Value Found (EI): {final_best_f_ei:.4f}")
```

#### 5. Experiment 2: CBO with Lower Confidence Bound (LCB) - Fixed Kappa
Applies the methodology from Deshwal et al. (2023) using LCB with fixed $\kappa$.

```python
# --- LCB Specific Acquisition Function and Optimization ---
def lower_confidence_bound(mu_pred: np.ndarray, sigma_pred: np.ndarray, kappa: float) -> np.ndarray:
    """Computes the Lower Confidence Bound (LCB) acquisition function."""
    sigma_pred = np.maximum(sigma_pred, 1e-9) 
    return mu_pred - kappa * sigma_pred 

def optimize_acqf_lcb(gp_model: GaussianProcessRegressor, scaler: MinMaxScaler, 
                      dictionary_A: np.ndarray, T_dim: int, q_batch_size: int, 
                      num_acqf_candidates: int, kappa_val: float) -> np.ndarray:
    """Optimizes the LCB acquisition function using HED."""
    m_embed_dim = dictionary_A.shape[0]
    candidate_u_vectors_np = np.random.randint(0, 2, size=(num_acqf_candidates, T_dim))
    embedded_candidates_np = embed_batch(candidate_u_vectors_np, dictionary_A)
    embedded_candidates_scaled_np = scaler.transform(embedded_candidates_np)
    mu_pred, std_pred = gp_model.predict(embedded_candidates_scaled_np, return_std=True)
    acq_values = lower_confidence_bound(mu_pred, std_pred, kappa=kappa_val)
    top_indices = np.argsort(acq_values)[-q_batch_size:][::-1]
    return candidate_u_vectors_np[top_indices, :]

# --- LCB (Fixed Kappa) BO Loop ---
print("\n--- Starting Experiment 2: CBO with LCB (Fixed Kappa) ---")
KAPPA_FIXED = 2.576 # Standard value for ~99% confidence

evaluated_U_np_list_lcb_fixed = []
evaluated_f_vals_lcb_fixed = []
train_Y_list_lcb_fixed = []

# Initialization
print(f"Generating {N_INITIAL_POINTS} initial random points for LCB (Fixed Kappa)...")
initial_candidates_lcb_fixed = []
while len(initial_candidates_lcb_fixed) < N_INITIAL_POINTS:
    U_init_candidate = np.random.randint(0, 2, size=T)
    if not any(np.array_equal(U_init_candidate, u) for u in initial_candidates_lcb_fixed):
        initial_candidates_lcb_fixed.append(U_init_candidate)

for U_init in initial_candidates_lcb_fixed:
    f_val = evaluate_objective(U_init, X_initial_schedule, v_star, convolutions, d, w)
    neg_f_val = -f_val
    evaluated_U_np_list_lcb_fixed.append(U_init)
    evaluated_f_vals_lcb_fixed.append(f_val)
    train_Y_list_lcb_fixed.append(neg_f_val)

best_obj_so_far_lcb_fixed = min(evaluated_f_vals_lcb_fixed) if evaluated_f_vals_lcb_fixed else float('inf')
print(f"Initial best objective value (LCB Fixed Kappa): {best_obj_so_far_lcb_fixed:.4f}")

# BO Iterations
for iteration in range(N_BO_ITERATIONS):
    iter_start_time = time.time()
    print(f"\n--- LCB (Fixed Kappa) Iteration {iteration + 1}/{N_BO_ITERATIONS} ---")
    current_dictionary_A_lcb_fixed = generate_diverse_random_dictionary(T, M_EMBED_DIM)
    if not evaluated_U_np_list_lcb_fixed: continue
    evaluated_U_np_array_lcb_fixed = np.array(evaluated_U_np_list_lcb_fixed)
    embedded_train_X_lcb_fixed = embed_batch(evaluated_U_np_array_lcb_fixed, current_dictionary_A_lcb_fixed)
    scaler_lcb_fixed = MinMaxScaler()
    if embedded_train_X_lcb_fixed.shape[0] > 0:
        embedded_train_X_scaled_lcb_fixed = scaler_lcb_fixed.fit_transform(embedded_train_X_lcb_fixed)
    else: embedded_train_X_scaled_lcb_fixed = embedded_train_X_lcb_fixed
    train_Y_for_fit_lcb_fixed = np.array(train_Y_list_lcb_fixed)
    if embedded_train_X_scaled_lcb_fixed.shape[0] > 0 and train_Y_for_fit_lcb_fixed.shape[0] == embedded_train_X_scaled_lcb_fixed.shape[0]:
        gp_model_lcb_fixed = get_fitted_model(embedded_train_X_scaled_lcb_fixed, train_Y_for_fit_lcb_fixed, M_EMBED_DIM)
    else: continue
    next_U_candidates_np_lcb_fixed = optimize_acqf_lcb(
        gp_model_lcb_fixed, scaler_lcb_fixed, current_dictionary_A_lcb_fixed, T, BATCH_SIZE_Q, NUM_ACQF_CANDIDATES, KAPPA_FIXED
    )
    for i_cand in range(next_U_candidates_np_lcb_fixed.shape[0]):
        next_U = next_U_candidates_np_lcb_fixed[i_cand, :]
        if any(np.array_equal(next_U, u) for u in evaluated_U_np_list_lcb_fixed): continue
        next_f = evaluate_objective(next_U, X_initial_schedule, v_star, convolutions, d, w)
        evaluated_U_np_list_lcb_fixed.append(next_U)
        evaluated_f_vals_lcb_fixed.append(next_f)
        train_Y_list_lcb_fixed.append(-next_f)
        if next_f < best_obj_so_far_lcb_fixed: best_obj_so_far_lcb_fixed = next_f
    print(f"Best objective (LCB Fixed Kappa) so far: {best_obj_so_far_lcb_fixed:.4f}")

# --- Results for LCB (Fixed Kappa) ---
print("\n--- LCB (Fixed Kappa) Optimization Finished ---")
if not evaluated_f_vals_lcb_fixed:
    final_best_f_lcb_fixed = float('inf'); final_best_U_lcb_fixed = None
else:
    final_best_idx_lcb_fixed = np.argmin(evaluated_f_vals_lcb_fixed)
    final_best_U_lcb_fixed = evaluated_U_np_list_lcb_fixed[final_best_idx_lcb_fixed]
    final_best_f_lcb_fixed = evaluated_f_vals_lcb_fixed[final_best_idx_lcb_fixed]
    print(f"Total evaluations (LCB Fixed Kappa): {len(evaluated_f_vals_lcb_fixed)}")
    print(f"Best Objective Value Found (LCB Fixed Kappa): {final_best_f_lcb_fixed:.4f}")
```

#### 6. Experiment 3: CBO with Lower Confidence Bound (LCB) - Increasing Kappa
Applies the methodology from Deshwal et al. (2023) using LCB with dynamic $\kappa$.

```python
# --- LCB (Increasing Kappa) BO Loop ---
print("\n--- Starting Experiment 3: CBO with LCB (Increasing Kappa) ---")
INITIAL_KAPPA_DYNAMIC = 3.75
KAPPA_INCREASE_FACTOR = 1.3
# MAX_KAPPA_DYNAMIC = 15.0 # Optional cap

evaluated_U_np_list_lcb_dyn = []
evaluated_f_vals_lcb_dyn = []
train_Y_list_lcb_dyn = []
current_kappa_dynamic = INITIAL_KAPPA_DYNAMIC

# Initialization
print(f"Generating {N_INITIAL_POINTS} initial random points for LCB (Increasing Kappa)...")
initial_candidates_lcb_dyn = []
while len(initial_candidates_lcb_dyn) < N_INITIAL_POINTS:
    U_init_candidate = np.random.randint(0, 2, size=T)
    if not any(np.array_equal(U_init_candidate, u) for u in initial_candidates_lcb_dyn):
        initial_candidates_lcb_dyn.append(U_init_candidate)

for U_init in initial_candidates_lcb_dyn:
    f_val = evaluate_objective(U_init, X_initial_schedule, v_star, convolutions, d, w)
    neg_f_val = -f_val
    evaluated_U_np_list_lcb_dyn.append(U_init)
    evaluated_f_vals_lcb_dyn.append(f_val)
    train_Y_list_lcb_dyn.append(neg_f_val)

best_obj_so_far_lcb_dyn = min(evaluated_f_vals_lcb_dyn) if evaluated_f_vals_lcb_dyn else float('inf')
print(f"Initial best objective value (LCB Increasing Kappa): {best_obj_so_far_lcb_dyn:.4f}")
print(f"Initial Kappa: {current_kappa_dynamic:.3f}")

# BO Iterations
for iteration in range(N_BO_ITERATIONS):
    iter_start_time = time.time()
    print(f"\n--- LCB (Increasing Kappa) Iteration {iteration + 1}/{N_BO_ITERATIONS} ---")
    current_dictionary_A_lcb_dyn = generate_diverse_random_dictionary(T, M_EMBED_DIM)
    if not evaluated_U_np_list_lcb_dyn: continue
    evaluated_U_np_array_lcb_dyn = np.array(evaluated_U_np_list_lcb_dyn)
    embedded_train_X_lcb_dyn = embed_batch(evaluated_U_np_array_lcb_dyn, current_dictionary_A_lcb_dyn)
    scaler_lcb_dyn = MinMaxScaler()
    if embedded_train_X_lcb_dyn.shape[0] > 0:
        embedded_train_X_scaled_lcb_dyn = scaler_lcb_dyn.fit_transform(embedded_train_X_lcb_dyn)
    else: embedded_train_X_scaled_lcb_dyn = embedded_train_X_lcb_dyn
    train_Y_for_fit_lcb_dyn = np.array(train_Y_list_lcb_dyn)
    if embedded_train_X_scaled_lcb_dyn.shape[0] > 0 and train_Y_for_fit_lcb_dyn.shape[0] == embedded_train_X_scaled_lcb_dyn.shape[0]:
        gp_model_lcb_dyn = get_fitted_model(embedded_train_X_scaled_lcb_dyn, train_Y_for_fit_lcb_dyn, M_EMBED_DIM)
    else: continue
    next_U_candidates_np_lcb_dyn = optimize_acqf_lcb(
        gp_model_lcb_dyn, scaler_lcb_dyn, current_dictionary_A_lcb_dyn, T, BATCH_SIZE_Q, NUM_ACQF_CANDIDATES, current_kappa_dynamic
    )
    improvement_found_in_batch = False
    for i_cand in range(next_U_candidates_np_lcb_dyn.shape[0]):
        next_U = next_U_candidates_np_lcb_dyn[i_cand, :]
        if any(np.array_equal(next_U, u) for u in evaluated_U_np_list_lcb_dyn): continue
        next_f = evaluate_objective(next_U, X_initial_schedule, v_star, convolutions, d, w)
        evaluated_U_np_list_lcb_dyn.append(next_U)
        evaluated_f_vals_lcb_dyn.append(next_f)
        train_Y_list_lcb_dyn.append(-next_f)
        if next_f < best_obj_so_far_lcb_dyn:
            best_obj_so_far_lcb_dyn = next_f
            old_kappa = current_kappa_dynamic
            current_kappa_dynamic *= KAPPA_INCREASE_FACTOR
            # current_kappa_dynamic = min(current_kappa_dynamic, MAX_KAPPA_DYNAMIC) # Optional cap
            improvement_found_in_batch = True
    print(f"Best objective (LCB Increasing Kappa) so far: {best_obj_so_far_lcb_dyn:.4f}")
    # print(f"Kappa for next iteration: {current_kappa_dynamic:.3f}")

# --- Results for LCB (Increasing Kappa) ---
print("\n--- LCB (Increasing Kappa) Optimization Finished ---")
if not evaluated_f_vals_lcb_dyn:
    final_best_f_lcb_dyn = float('inf'); final_best_U_lcb_dyn = None
else:
    final_best_idx_lcb_dyn = np.argmin(evaluated_f_vals_lcb_dyn)
    final_best_U_lcb_dyn = evaluated_U_np_list_lcb_dyn[final_best_idx_lcb_dyn]
    final_best_f_lcb_dyn = evaluated_f_vals_lcb_dyn[final_best_idx_lcb_dyn]
    print(f"Total evaluations (LCB Increasing Kappa): {len(evaluated_f_vals_lcb_dyn)}")
    print(f"Best Objective Value Found (LCB Increasing Kappa): {final_best_f_lcb_dyn:.4f}")
    print(f"Final Kappa value reached: {current_kappa_dynamic:.3f}")
```

## Results

The initial schedule, derived using the Bailey-Welch method (Kaandorp and Koole, 2007), serves as a baseline. The objective function $C(\mathbf{x})$ combines Expected Waiting Time ($EWT$) and Expected Staff Penalty ($ESP$). Lower values of $C(\mathbf{x})$ are preferable. Each experiment consisted of $N_{INITIAL} = 20$ initial random evaluations followed by $N_{ITERATIONS} = 25$ Bayesian optimization iterations, with $BATCH\_SIZE_q = 5$ evaluations per iteration, totaling approximately $20 + 25 \times 5 = 145$ evaluations per experiment. The optimization operates on the binary perturbation vector $\mathbf{U}$, using the HED embedding (Deshwal et al., 2023).

The key performance metric is the best (minimum) objective function value found.

### Experiment 1: CBO with Expected Improvement (EI)
-   **Initial Best Objective (after random search)**: $73.6389$
-   **Final Best Objective Found**: $68.2348$
-   **Total Evaluations**: $145$
-   **Best Perturbation Vector $\mathbf{U}_{EI}^*$**: `[0 0 0 1 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0 1]`
-   **Resulting Optimal Schedule $\mathbf{x}_{EI}^*$**: `[2 1 1 0 1 0 2 0 0 1 1 1 0 1 0 1 1 0 1 7]`

### Experiment 2: CBO with Lower Confidence Bound (LCB) - Fixed Kappa ($\kappa = 2.576$)
-   **Initial Best Objective (after random search)**: $68.0201$
-   **Final Best Objective Found**: $66.9201$
-   **Total Evaluations**: $143$
-   **Best Perturbation Vector $\mathbf{U}_{LCB\_fixed}^*$**: `[0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1]`
-   **Resulting Optimal Schedule $\mathbf{x}_{LCB\_fixed}^*$**: `[2 1 1 0 1 1 1 0 1 0 1 1 0 1 1 0 1 1 0 7]`

### Experiment 3: CBO with Lower Confidence Bound (LCB) - Increasing Kappa
-   **Initial Best Objective (after random search)**: $74.0010$
-   **Final Best Objective Found**: $68.0497$
-   **Total Evaluations**: $145$
-   **Best Perturbation Vector $\mathbf{U}_{LCB\_dyn}^*$**: `[0 0 0 1 0 0 1 1 0 0 0 1 1 0 1 1 0 1 0 1]`
-   **Resulting Optimal Schedule $\mathbf{x}_{LCB\_dyn}^*$**: `[2 1 1 0 1 1 1 0 0 1 1 1 0 1 1 0 1 0 1 7]`
-   **Initial $\kappa$**: $3.750$
-   **Final $\kappa$**: $39.767$

### Summary of Best Objectives
| Experiment                       | Best Objective $C(\mathbf{x}^*)$ |
|----------------------------------|-----------------------------------|
| CBO with EI                      | $68.2348$                         |
| CBO with LCB (Fixed $\kappa=2.576$) | $66.9201$                         |
| CBO with LCB (Increasing $\kappa$) | $68.0497$                         |

## Discussion

The experiments aimed to compare three CBO strategies, leveraging the HED embedding technique (Deshwal et al., 2023), for optimizing the outpatient appointment scheduling problem formulated by Kaandorp and Koole (2007). All three methods successfully improved upon their respective initial random search results, demonstrating the applicability of BO with HED to this combinatorial problem.

1.  **Performance Comparison**: The CBO strategy employing LCB with a fixed $\kappa = 2.576$ yielded the lowest objective function value ($66.9201$), indicating the best performance among the tested methods for this specific problem instance and parameter settings. EI achieved a final objective of $68.2348$, while LCB with increasing $\kappa$ reached $68.0497$.

2.  **Hypothesis Evaluation**:
    -   Hypothesis 1 was supported: All CBO strategies found significantly better schedules than the initial random points.
    -   Hypothesis 2 received partial support: LCB with fixed $\kappa$ outperformed EI, but LCB with increasing $\kappa$ did not significantly outperform EI.
    -   Hypothesis 3 was not supported: The dynamic $\kappa$ strategy, despite adapting its exploration parameter substantially (from $3.750$ to $39.767$), did not achieve the best result, being outperformed by the fixed $\kappa$ LCB.

3.  **Exploration vs. Exploitation**: The fixed $\kappa$ LCB strategy appears to strike an effective balance for this problem. The dynamic $\kappa$ strategy, becoming increasingly explorative upon finding improvements, might have explored too broadly after initial gains, potentially moving away from the most promising regions identified earlier. The EI strategy's focus on immediate expected improvement was less effective than the fixed LCB's balanced approach. The effectiveness of the HED embedding, combined with ARD in the GP kernel (as discussed by Deshwal et al., 2023), likely played a crucial role in enabling the GP to model the complex objective landscape effectively.

4.  **Computational Effort**: All methods required a similar number of expensive objective function evaluations (~145). The main computational overhead per iteration, beyond function evaluation, involves HED embedding, GP fitting, and acquisition function optimization.

5.  **Limitations and Future Work**:
    -   The optimality guarantee mentioned by Kaandorp and Koole (2007) applies to their specific local search algorithm operating directly on the schedule space $\mathcal{F}$, leveraging multimodularity. Our BO approach operates on the perturbation vector space $\mathbf{U}$ via HED embeddings. While BO aims for global optimization, it doesn't inherit the same theoretical guarantee of finding the global optimum as the original local search, especially given the stochastic nature of GP modeling and acquisition function optimization.
    -   The performance is likely sensitive to BO hyperparameters (dictionary size $m$, $\kappa$ values, number of candidates for acquisition optimization).
    -   The mock implementation of `functions.py` needs replacement with actual scheduling logic for accurate results.
    -   Further investigation into different dictionary construction methods (e.g., binary wavelets as mentioned in Deshwal et al., 2023) or adaptive $\kappa$ schedules could be beneficial.

In conclusion, applying CBO with HED embeddings appears promising for this scheduling problem. The LCB acquisition function with a fixed, well-chosen $\kappa$ demonstrated the best performance in this study.

## Timeline

-   **Experiment Setup and Code Implementation**: (Specify Date Range)
-   **Initial Parameter Tuning (if any)**: (Specify Date Range)
-   **Execution of Experiment 1 (EI)**: (Specify Date)
-   **Execution of Experiment 2 (LCB Fixed Kappa)**: (Specify Date)
-   **Execution of Experiment 3 (LCB Increasing Kappa)**: (Specify Date)
-   **Results Analysis and Report Compilation**: (Specify Date Range)

## References

-   Deshwal, A., Ament, S., Balandat, M., Bakshy, E., Doppa, J. R., & Eriksson, D. (2023). Bayesian Optimization over High-Dimensional Combinatorial Spaces via Dictionary-based Embeddings. *Proceedings of the 26th International Conference on Artificial Intelligence and Statistics (AISTATS)*, PMLR 206:5141-5168.
-   Kaandorp, G. C., & Koole, G. (2007). Optimal outpatient appointment scheduling. *Health Care Management Science*, 10(3), 217–229. https://doi.org/10.1007/s10729-007-9015-x
-   (Placeholder for Bailey-Welch algorithm reference, e.g., Bailey, N. T. J. (1952). A study of queues and appointment systems in hospital out-patient departments, with special reference to waiting-times. *Journal of the Royal Statistical Society: Series B (Methodological)*, 14(2), 185-199.)
-   (Placeholder for Gaussian Process literature, e.g., Rasmussen, C. E., & Williams, C. K. I. (2006). *Gaussian Processes for Machine Learning*. MIT Press.)

```

Ik heb de verwijzingen naar (Kaandorp and Koole, 2007) en (Deshwal et al., 2023) toegevoegd in de secties Objective, Background, Methodology, Discussion en References. De tekst is aangepast om de bijdragen van elk paper aan het huidige experiment te verduidelijk